In [ ]:
import pandas as pds
import numpy as np #importamos las paqueterias pandas y numpy

from google.colab import drive
drive.mount('/content/drive')

class Grafica(): #Creamos la clase gráfica así mismo definimos a la matriz inicial
  def __init__(self, nodos, arcos):
    self.matriz = np.zeros((len(nodos),len(nodos)))
    for x in arcos:
      self.matriz[x[0],x[1]] = 1 # definimos a los arcos mediante los nodos

  def __str__(self):
    return str(self.matriz)

  def agregar_arco(self, tupla): #definimos agregar arcos con el valor 1
    self.matriz[tupla[0], tupla[1]] = 1

  def quitar_arco(self, tupla): #definimos quitar arcos con el valor 0
    self.matriz[tupla[0], tupla[1]] = 0

  def num_nodos(self): #total de nodos será igual a la longitud de la matriz
    return len(self.matriz)

  def num_arcos(self): #total de arcos será la suma de valores 1 que representan los arcos
    return int(self.matriz.sum())

  def agregar_nodo(self):
    vertical = np.zeros((len(self.matriz),1))
    horizontal = np.zeros((1,len(self.matriz)+1))
    self.matriz = np.vstack((np.hstack((self.matriz,vertical)),
                             horizontal))

  def quitar_nodo(self, nodo):
    self.matriz = np.delete(self.matriz, (nodo), axis=0)
    self.matriz = np.delete(self.matriz, (nodo), axis=1)

  def revisar_conexidad(self):
    for i in range(len(self.matriz)):
      if self.matriz[i,:].sum() == 0:
        return False
    return True

  def revisar_ciclicidad(self):
    for i in range(len(self.matriz)):
      if self.matriz[i,i] == 1:
        return True
    return False

  def revisar_adyacencia(self, arco):
    if self.matriz[arco[0], arco[1]] == 1:
      return True
    return False

  def num_adyacentes(self, nodo):
    return self.matriz[nodo,:].sum()

  def num_conectados(self, nodo):
    return self.matriz[:,nodo].sum()

def construccion_pds(ruta_del_archivo): # definimos las bases para obtener la grafica
  df = pds.read_excel(ruta_del_archivo)
  Nodos = (list(range(len(df["Index"]))))
  x = list(df["Cited by"])
  Arcos = []
  for i in range(len(x)):
    for j in x[i].split(','):
      Arcos.append((int(j) - 1,i))
  return Grafica(Nodos, Arcos) #recibimos a la gráfica

class red(Grafica): #clase red que hereda de gGráfica

  def __init__(self, nodos, arcos):
    super().__init__(nodos, arcos) #llama a l clase suprema

  def matriz_pesos(self): #definimos a la matriz de pesos de los arcos
    pesos = np.zeros_like(self.matriz)
    for i in range(len(self.matriz)):
      pesos[i, :] = self.matriz[i, :]/self.matriz[i,:].sum()
      return pesos

class Pagerank(red): #definimos a la clase del algoritmo pagerank
  def __init__(self, nodos, arcos, epsilon=1e-6): #añadimos una nueva característica llamada epsilon
    super().__init__(nodos, arcos)
    self.epsilon = epsilon
  def __call__(self, d): # llamamos a la función evaluadora, que evaluará a d
    n= self.num_nodos()
    vector_inicial = np.ones(n)/self.num_nodos()
    pi_m = vector_inicial
    P = self.matriz_pesos()
    U = np.ones((n,n))/self.num_nodos()
    pi_m1 = pi_m(d*P + (1-d)*U) #nueva forma de pi
    return pi_m1
if __name__ == "__main__": #usamos el archivo que importamos de drive para obtener los datos
  df = pds.read_excel("/content/drive/MyDrive/Colab Notebooks/Web.xlsx")
  matrix = construccion_pds("/content/drive/MyDrive/Colab Notebooks/Web.xlsx").matriz
  print(matrix) # matrix representará a la matriz base de la grafica
  pesos = np.zeros_like(matrix)
  for i in range(len(matrix)):
      pesos[i, :] =  matrix[i, :]/matrix[i,:].sum()
  print(pesos) # volvemos a usar pesos mediante matrix

  pi = np.ones((1, len(pesos)))/len(pesos)
  pi_1 = pi@pesos
  while np.linalg.norm(pi - pi_1) > 1e-6: # mientras la distancia de pi a pi_1 sea mayor a 1e-6 continuará
    pi = pi_1
    pi_1 = pi@pesos
  print(pi) #representa los valores en los que ya se cumple la condición del while
  print("1.- se deduce que la pagina mas importante es la 26")

 # pregunta 2..... teia la idea pero se me complico y no supe como proseguir despues de obtener lo siguiente
 # sitios = df["Website"].tolist() # recuperamos a los websites en una lista
 # sitios_ru = sitios_ru = [sitio for sitio in sitios if sitio.endswith('.ru')]
 # recuperamos solo los que tengan temr
# pregunta 3..... no entendi como evaluar d con la funcion call

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[[0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 1.]
 [1. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 1.]
 [0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 1.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 1.]
 [0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

AttributeError: 'DataFrame' object has no attribute 'matriz'